In [ ]:
import mysql.connector
import random
from datetime import datetime, timedelta

# Połączenie z bazą danych
connection = mysql.connector.connect(
    host="giniewicz.it",
    user="team19",
    password="te@mzaiq",
    database="team19"
)

cursor = connection.cursor()

# Zapytanie SQL do wstawiania danych do tabeli Oceny
insert_opinion_query = """
INSERT INTO Oceny (id_rezerwacje, Ocena, Data_oceny) 
VALUES (%s, %s, %s);
"""

# Pobranie zakończonych rezerwacji, które zostały zrealizowane
cursor.execute("""
    SELECT 
        r.id_rezerwacje, 
        rw.data_rozpoczecia, 
        rw.data_zakonczenia
    FROM Rezerwacje r
    INNER JOIN Transakcje_finansowe tf ON r.id_transakcje = tf.id_transakcje
    INNER JOIN Realizowane_wycieczki rw ON r.id_wycieczki = rw.id_wycieczki
    WHERE tf.Zrealizowano = 1
""")
rezerwacje = cursor.fetchall()

# Pobranie już ocenionych rezerwacji
cursor.execute("SELECT id_rezerwacje FROM Oceny")
ocenione_rezerwacje = {row[0] for row in cursor.fetchall()}

# Generowanie ocen dla rezerwacji
for (id_rezerwacje, data_rozpoczecia, data_zakonczenia) in rezerwacje:
    if id_rezerwacje not in ocenione_rezerwacje:
        ocena = random.randint(1, 10)  # Ocena w skali od 1 do 10
        data_oceny = data_zakonczenia + timedelta(days=random.randint(1, 30))  # Data oceny jest późniejsza

        # Wstawianie danych do tabeli Oceny
        cursor.execute(insert_opinion_query, (id_rezerwacje, ocena, data_oceny.strftime('%Y-%m-%d')))

# Zatwierdzenie zmian w bazie danych
connection.commit()

# Zamknięcie kursora i połączenia
cursor.close()
connection.close()
